In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import json
import numpy as np

In [ ]:
df_bohdan = pd.read_csv('final_dataset.csv')
df_sofiia = pd.read_csv('rdi1_reduced/rdi.csv')
df_names = pd.read_csv('archive-diploma-kaggle/all_dist_villages.csv')

In [ ]:
df_names = df_names[["admin4Name", "admin4Na_1", "admin4Pcod"]]

In [ ]:
df_orig = df_bohdan.merge(df_sofiia, on='admin4Pcod', how='inner', suffixes=('_1', ''))
df_orig = df_orig.merge(df_names, on='admin4Pcod', how='inner', suffixes=('_1', ''))

In [ ]:
df = df_orig[['X', 'Y', 'RDI', 'admin4Name', "admin4Pcod"]]
df = df.rename(columns={"admin4Pcod": "Postal code", "admin4Name": "Locality"})

df['RDI'] = np.round(df['RDI'], 3)

bins = [0, 0.65, 0.75, 0.85, 0.90, 1.00]
labels = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']
labels_legend = ['0.00-0.65 (Very Low)', '0.65-0.75 (Low)', '0.75-0.85 (Moderate)', '0.85-0.90 (High)', '0.90-1.00 (Very High)']
df['RDI category'] = pd.cut(df['RDI'], bins=bins, labels=labels, include_lowest=True)
df['RDI legend'] = pd.cut(df['RDI'], bins=bins, labels=labels_legend, include_lowest=True)
df.sort_values('RDI', ascending=False, inplace=True)

custom_data = df[['Locality', "Postal code", 'RDI']]

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import json

def plot_borders(dst_fig):
    fig = go.Figure()

    # Load GeoJSON geometry data in X, Y format
    with open('adm1.geojson') as f:
        geojson = json.load(f)

    # Loop through each feature in the GeoJSON
    for feature in geojson['features']:
        # Extract the coordinates for each polygon
        coordinates = feature['geometry']['coordinates']
        
        # If the geometry is a polygon
        if feature['geometry']['type'] == 'Polygon':
            # Extract the outer ring of the polygon
            x_coords, y_coords = zip(*coordinates[0])  # Only take the first ring
            
            # Add the polygon's boundary as lines to the figure
            fig.add_trace(go.Scattergl(
                x=x_coords,
                y=y_coords,
                mode='lines',  # Lines only
                line=dict(color='black'),  # Color of the lines
                name=None,
                showlegend=False,
                    hoverinfo='none'
            ))

        # If the geometry is a MultiPolygon
        elif feature['geometry']['type'] == 'MultiPolygon':
            for polygon in coordinates:
                x_coords, y_coords = zip(*polygon[0])  # Take the first ring of each polygon
                
                # Add the polygon's boundary as lines to the figure
                fig.add_trace(go.Scattergl(
                    x=x_coords,
                    y=y_coords,
                    mode='lines',  # Lines only
                    line=dict(color='black'),  # Color of the lines
                    name=None,
                    showlegend=False,
                    hoverinfo='none'
                ))

    # Remove interactive elements
    fig.update_traces(hoverinfo='none', selector=dict(type='scatter'))  # Disable hover info

    for trace in fig.data:
        dst_fig.add_trace(trace)

def plot_points(dst_fig):
    colors = { '0.00-0.65 (Very Low)': '#d7191c', '0.65-0.75 (Low)': '#f5a153', '0.75-0.85 (Moderate)': '#f7ff0a', '0.85-0.90 (High)': '#8acc62', '0.90-1.00 (Very High)': '#1a9641' }

    # Create scatter plot using plotly express
    scatter_fig = px.scatter(df, x='X', y='Y', color='RDI legend', color_discrete_map=colors, hover_data={
        'X': False,  # Hide X
        'Y': False,  # Hide Y
        'RDI category': True,  # Show RDI category
        'Locality': True,  # Show Locality
        'Postal code': True,  # Show Postal code
        'RDI': True  # Show RDI
    })

    hovertemplate = "<br>".join([
        "<b>%{customdata[1]}</b>",
        "%{customdata[2]}",
        "RDI: %{customdata[3]} (%{customdata[0]})<extra></extra>",
    ])

    scatter_fig.update_traces(
        marker=dict(size=4),
        hovertemplate=hovertemplate,
        # showlegend=False,
    )

    scatter_fig.update_xaxes(showgrid=False, zeroline=False)
    scatter_fig.update_yaxes(showgrid=False, zeroline=False)

    # Add scatter points to the main figure
    for trace in scatter_fig.data:
        dst_fig.add_trace(trace)


# Create the figure
fig = go.Figure()

# plot_borders(fig)
plot_points(fig)
plot_borders(fig)
fig.update_traces(line={'width': 1})  # Disable hover info


fig.update_layout(
    # title='Sample scatter plot',
    plot_bgcolor="white",
    xaxis_title=None,
    xaxis_showticklabels=False,
    yaxis_title=None,
    yaxis_showticklabels=False,
    showlegend=True,
    margin=dict(l=0,r=0,b=0,t=24),
    legend=dict(
        title_text='Rural development index',
        title_font=dict(size=16, color='black'),
        x=0.1,  # Position x at the left
        y=0.1,  # Position y at the bottom
        xanchor='left',  # Anchor the x position to the left
        yanchor='bottom',  # Anchor the y position to the bottom
        orientation='v',
        bgcolor='rgba(255, 255, 255, 0)',  # Transparent background
        bordercolor='rgba(255, 255, 255, 0)',  # Optional: remove border
        itemsizing='constant'
    ),
)

# Show the figure
fig.show()
fig.write_html('rdi_mcdm_interactive.html')